In [ ]:
import datetime
import time
import requests
import pandas as pd
import numpy as np

For getting this data from AEMET you need to get an API key, which can be requested for free here: https://opendata.aemet.es

In [ ]:
i = 0
headers = {'cache-control': "no-cache"}
params = {
    "api_key": "API_KEEY"
}
base_url = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos'

weather_df = pd.DataFrame()
start_date = datetime.date(2006, 10, 8)
end_date = datetime.date(2017, 9, 25)

while start_date < end_date:
    if i == 20:
        # To avoid raising the limit
        time.sleep(60)
        i = 0
    else:
        i += 1
    # Get week data
    end_week = start_date + datetime.timedelta(days=6)
    url = f'{base_url}/fechaini/{start_date.strftime("%Y-%m-%dT00:00:00UTC")}/fechafin/{end_week.strftime("%Y-%m-%dT23:59:59UTC")}/todasestaciones'
    response = requests.request("GET", url, headers=headers, params=params)
    data_url = response.json()['datos']
    # Second call for getting the data
    data_response = requests.request("GET", data_url)
    week_df = pd.DataFrame(data_response.json())

    # data cleaning and grouping
    week_df['week'] = f'{start_date.isocalendar()[1]}-{start_date.isocalendar()[0]}'
    start_date = start_date + datetime.timedelta(days=7)
    week_df = week_df[['week', 'provincia', 'tmin', 'tmax', 'tmed', 'prec', 'racha']]
    week_df = week_df.dropna()
    week_df['tmin'] = week_df['tmin'].apply(lambda x: float(x.replace(',', '.')))
    week_df['tmax'] = week_df['tmax'].apply(lambda x: float(x.replace(',', '.')))
    week_df['tmed'] = week_df['tmed'].apply(lambda x: float(x.replace(',', '.')))
    week_df['prec'] = week_df['prec'].apply(
        lambda x: float(x.replace(',', '.')) if ',' in x else None)
    week_df['racha'] = week_df['racha'].apply(lambda x: float(x.replace(',', '.')))
    week_df = week_df.dropna()
    week_df = week_df.groupby(
        ['week', 'provincia'], as_index=False).agg({
            'tmin': 'min',
            'tmax': 'max',
            'tmed': 'mean',
            'prec': 'sum',
            'racha': 'max'
        })
    weather_df = weather_df.append(week_df)

In [ ]:
weather_df.to_hdf('data/weather_data.hdf', 'data')